In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/data/data-20231115T183312Z-001.zip

Archive:  /content/drive/MyDrive/data/data-20231115T183312Z-001.zip
replace data/valid_Y.npy? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.210 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.7/78.2 GB disk)


In [ ]:
import torch
import numpy as np
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor, Resize
from PIL import Image

class CustomYOLODataset(Dataset):
    def __init__(self, images_file, labels_file, bboxes_file, target_size=(640, 640)):
        self.images = np.load(images_file, allow_pickle=True)
        self.labels = np.load(labels_file, allow_pickle=True)
        self.bboxes = np.load(bboxes_file, allow_pickle=True)
        self.transform = ToTensor()
        self.resize = Resize(target_size)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.fromarray(self.images[idx])

        # Convert grayscale to RGB if needed
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # Resize the image to the target size
        image = self.resize(image)

        label = torch.tensor(self.labels[idx])
        bbox = torch.tensor(self.bboxes[idx])

        # Apply YOLO-specific preprocessing here, if needed

        image = self.transform(image)

        return image, label, bbox

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from ultralytics import YOLO
from torch.nn import CrossEntropyLoss, SmoothL1Loss

classification_loss_fn = CrossEntropyLoss()
localization_loss_fn = SmoothL1Loss()


# Paths to your training data files
train_images_file = '/content/data/train_X.npy'
train_labels_file = '/content/data/train_Y.npy'
train_seg_file = '/content/data/train_seg.npy'

# Paths to your validation data files
valid_images_file = '/content/data/valid_X.npy'
valid_labels_file = '/content/data/valid_Y.npy'
valid_seg_file = '/content/data/valid_seg.npy'

# Create instances of the custom segmentation datasets
train_dataset = CustomYOLODataset(train_images_file, train_labels_file, train_seg_file)
valid_dataset = CustomYOLODataset(valid_images_file, valid_labels_file, valid_seg_file)

# Define DataLoaders
batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# Instantiate the YOLO-based segmentation model
model = model = YOLO('yolov8n-seg.pt') # Replace this with your YOLO-based segmentation model architecture

# Define loss function and optimizer
loss_fn = CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1  # Adjust as needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        images, labels, seg_masks = batch
        images = images.to(device)
        labels = labels.to(device)
        seg_masks = seg_masks.to(device)

        # Forward pass
        outputs = model(images)
        #print(outputs)

        boxes = outputs[0].boxes
        for box in boxes:

        # Extract predicted bounding box coordinates and class probabilities
          pred_bboxes = box.xyxy[:, :4]
          pred_class_probs = box.xyxy[:, 4]

          # Extract ground truth bounding box coordinates and class labels
          gt_bboxes = labels[:, :, 1:]
          gt_class_labels = labels[:, :, 0].long()

          # Calculate classification loss using cross-entropy loss
          classification_loss = classification_loss_fn(pred_class_probs, gt_class_labels)

          # Calculate localization loss using smooth L1 loss
          localization_loss = localization_loss_fn(pred_bboxes, gt_bboxes)

          # Total loss
          total_loss = classification_loss + localization_loss

        # Backward pass and optimization
          optimizer.zero_grad()
          total_loss.backward()
          optimizer.step()


#         #output_tensor = outputs[0]

#         # Reshape the output if needed
#         #output_tensor = output_tensor.view(output_tensor.size(0), -1)

#         # Calculate segmentation loss using cross-entropy loss
#         segmentation_loss = loss_fn(outputs, labels)

#         # Backward pass and optimization
#         optimizer.zero_grad()
#         segmentation_loss.backward()
#         optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        total_val_loss = 0
        for val_batch in valid_dataloader:
            val_images, val_labels, val_seg_masks = val_batch
            val_images = val_images.to(device)
            val_labels = val_labels.to(device)
            val_seg_masks = val_seg_masks.to(device)

            # Forward pass for validation
            val_outputs = model(val_images)
            boxes = outputs[0].boxes
            for box in boxes:

            # Extract predicted bounding box coordinates and class probabilities
              val_pred_bboxes = box.xyxy[:, :4]
              val_pred_class_probs = box.xyxy[:, 4]

              # Extract ground truth bounding box coordinates and class labels
              val_gt_bboxes = val_labels[:, :, 1:]
              val_gt_class_labels = val_labels[:, :, 0].long()

              # Calculate classification loss for validation
              val_classification_loss = classification_loss_fn(val_pred_class_probs, val_gt_class_labels)

              # Calculate localization loss for validation
              val_localization_loss = localization_loss_fn(val_pred_bboxes, val_gt_bboxes)

              # Total validation loss
              total_val_loss += val_classification_loss + val_localization_loss

    avg_val_loss = total_val_loss / len(valid_dataloader)

    print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {total_loss.item():.4f}, Validation Loss: {avg_val_loss.item():.4f}')

# Save the trained model if needed
torch.save(model.state_dict(), 'yolo_model.pth')

            #val_output_tensor = val_outputs[0]

            # Reshape the output if needed
            #val_output_tensor = val_output_tensor.view(val_output_tensor.size(0), -1)


            # Calculate segmentation loss for validation
#             val_segmentation_loss = loss_fn(val_outputs, val_labels)

#             # Total validation loss
#             total_val_loss += val_segmentation_loss

#     avg_val_loss = total_val_loss / len(valid_dataloader)

#     print(f'Epoch [{epoch + 1}/{num_epochs}], Train Loss: {segmentation_loss.item():.4f}, Validation Loss: {avg_val_loss.item():.4f}')

# # Save the trained model if needed
# torch.save(model.state_dict(), 'yolo_segmentation_model.pth')


Streaming output truncated to the last 5000 lines.
0: 640x640 (no detections), 1: 640x640 (no detections), 2: 640x640 (no detections), 3: 640x640 (no detections), 4: 640x640 (no detections), 5: 640x640 (no detections), 6: 640x640 (no detections), 7: 640x640 (no detections), 8: 640x640 (no detections), 9: 640x640 (no detections), 10: 640x640 (no detections), 11: 640x640 (no detections), 12: 640x640 (no detections), 13: 640x640 (no detections), 14: 640x640 (no detections), 15: 640x640 (no detections), 107.7ms
Speed: 0.0ms preprocess, 6.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 1: 640x640 (no detections), 2: 640x640 (no detections), 3: 640x640 (no detections), 4: 640x640 (no detections), 5: 640x640 (no detections), 6: 640x640 (no detections), 7: 640x640 (no detections), 8: 640x640 (no detections), 9: 640x640 (no detections), 10: 640x640 (no detections), 11: 640x640 (no detections), 12: 640x640 (no detections), 13: 640x640 (no detecti

NameError: ignored